In [1]:
from preprocess import impute
import pandas as pd
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
train = pd.read_csv('train.csv').set_index('Id')
test = pd.read_csv('test.csv').set_index('Id')
all_data = pd.concat([train, test])
train_c, encodedDic = impute(all_data, False)
onehot, encodedDic1 = impute(all_data, True)
target = train_c['SalePrice'][:1460]
train_len = train_c[:1460]
test_len = train_c[1460:]
train_oh = onehot[:1460]
test_oh = onehot[1460:]
train_len = train_len.drop(["SalePrice"], axis = 1)
test_len = test_len.drop(['SalePrice'], axis = 1)
train_oh = train_oh.drop(["SalePrice"], axis = 1)
test_oh = test_oh.drop(["SalePrice"], axis = 1)
target = train['SalePrice']

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



# SEDUCTIVE MODELING

In [2]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error 
from xgboost import XGBRegressor
from scipy import stats
from scipy.stats import norm, skew
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV # search for the best lambda
from sklearn import linear_model

# XGB
    -label encoded data
    -log(y)
    -BoxCox(x)

In [3]:
temp_len = pd.concat([train_len, test_len])

num_data = temp_len.dtypes[train_len.dtypes != 'object'].index
skewed_data = temp_len[num_data].apply(lambda x: skew(x))
skewed = skewed_data[abs(skewed_data) > 0.75]

from scipy.special import boxcox1p
for i in skewed.index:
    temp_len[i] = boxcox1p(temp_len[i], 0.15)
    
train_lt = temp_len[:1460]
test_lt = temp_len[1460:]
X = train_lt
y = np.log(target)

In [4]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

In [5]:
model = XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =42, nthread = -1)

In [6]:
model_xgb = model.fit(train_X, train_y)

In [7]:
y_predict = np.expm1(model.predict(train_X))

In [8]:
def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions - targets) ** 2))

In [9]:
rmse(train_y, y_predict)

195202.02473435635

In [10]:
transform_xgb = np.expm1(model.predict(test_lt))
#kaggle score: 0.12772

sub = pd.DataFrame()
sub['Id'] = test.index
sub['SalePrice'] = transform_xgb
sub.set_index('Id')
sub.to_csv('m_xgb_transform.csv')

In [11]:
from sklearn.model_selection import GridSearchCV # search for the best lambda
from sklearn import linear_model

ridge = linear_model.Ridge(normalize=True) # create a ridge regression instance

# find the best alpha (lambda) for ridge
grid_param = [{'alpha': np.logspace(-4, 2, 100)}]
para_search_ridge = GridSearchCV(estimator=ridge, param_grid=grid_param, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
para_search_ridge.fit(X, y)

print(para_search_ridge.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(para_search_ridge.best_score_)))
# fit best ridge equation to all train data
best_ridge_y = para_search_ridge.best_estimator_.predict(X)
print("RMSE: ", np.sqrt(np.mean((y-best_ridge_y)**2)))

{'alpha': 0.8697490026177834}
Lowest RMSE found:  0.1555939818637338
RMSE:  0.1380056093453742


In [ ]:

print("RMSE: ", np.sqrt(np.mean((y_log-best_ridge_y)**2)))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=42))

In [ ]:
#model.fit(train_X, train_y, early_stopping_rounds=5, 
#             eval_set=[(test_X, test_y)], verbose=False)

In [ ]:
#prediction_test = model.predict(test_X)

In [ ]:
ENet.fit(train_X, train_y)

In [ ]:
train_enet = np.expm1(ENet.predict(test_X))

In [ ]:
#train_enet

In [ ]:
rmse(train_enet, test_y)

In [ ]:
test_lt

In [ ]:
cross_val_score(model, X, y, scoring='neg_mean_absolute_error')

In [ ]:
cross_val_score(ENet,X, y, scoring='neg_mean_absolute_error')

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test.index
sub['SalePrice'] = transform_xgb
sub.set_index('Id')
sub.to_csv('m_xgb_transform.csv')